In [2]:
import tensorflow as tf

import numpy as np
import os
import time
import random

In [3]:
# Maybe need to alter this to function with S3.
path_to_file = "poems_final.txt"


In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='latin-1')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 5085301 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])


The Pontalba Apartments in the View-Master
and the cardboard cathedral as if trapped in the dream
twenty years early, the whole a furious search
as if for a reason. Still, it's sex that spoils it, isn't it?

Jackson Square was the smallest sufficien


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

134 unique characters


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  '+' :  12,
  ',' :  13,
  '-' :  14,
  '.' :  15,
  '/' :  16,
  '0' :  17,
  '1' :  18,
  '2' :  19,
  ...
}


In [9]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))


'\nThe Pontalba' ---- characters mapped to int ---- > [ 0 51 69 66  1 47 76 75 81 62 73 63 62]


In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])



T
h
e
 


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\nThe Pontalba Apartments in the View-Master\nand the cardboard cathedral as if trapped in the dream\ntw'
"enty years early, the whole a furious search\nas if for a reason. Still, it's sex that spoils it, isn'"
't it?\n\nJackson Square was the smallest sufficient landscape,\nbut that was before, and now the gates a'
're locked\nat sundown, and the smell of the river fails,\nfalls ever backward. Some of us lived there, '
'though:\nWhat shall we say who have knowledge\ncarried to the heart? Back then those flat facades\nshone'


In [12]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\nThe Pontalba Apartments in the View-Master\nand the cardboard cathedral as if trapped in the dream\nt'
Target data: 'The Pontalba Apartments in the View-Master\nand the cardboard cathedral as if trapped in the dream\ntw'


In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 0 ('\n')
  expected output: 51 ('T')
Step    1
  input: 51 ('T')
  expected output: 69 ('h')
Step    2
  input: 69 ('h')
  expected output: 66 ('e')
Step    3
  input: 66 ('e')
  expected output: 1 (' ')
Step    4
  input: 1 (' ')
  expected output: 47 ('P')


In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [18]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 134) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           34304     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 134)           137350    
Total params: 4,109,958
Trainable params: 4,109,958
Non-trainable params: 0
_________________________________________________________________


In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
sampled_indices

array([ 65,  26,  35, 105,   6,  67,  51,  82,  23,  72,  86,  89,  21,
       118,  54,  85,  17,  84, 126,  54, 107, 131,  94, 133,  25, 110,
        58, 115,  17,  12,  49,  49,  43,  25,  47,  31,  27,  53,  68,
        39,  92,  73, 133,  57,  42, 133,  85,  30,  19,  57,  47,  28,
        86,  88,  55, 103, 126, 132, 126,  55,  20,  43,  25,  97,  15,
        18,  95,  43, 119,   0,  69,  62,  37, 101,  35,  85, 106,  38,
        64,  36,  46,  34,  35, 124,  43,  43, 119,  89,   1,  44, 130,
        79,  35, 119,  32, 101, 122, 100,  31,  56])

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '"\n"\n\napril is a dog\'s dream\n\nthe soft grass is growing\n\nthe sweet breeze is blowing\n\nthe air all ful'

Next Char Predictions: 
 'd9D¦%fTu6ky|4¶Wx0wÅW¨é\x83ó8«[±0+RRL8P?:VgH\x81lóZKóx=2ZP;y{X¤ÅïÅX3L8\x8b.1\x86L¹\nhaF¡Dx§GcEOCD¿LL¹| MèrD¹A¡¼\xa0?Y'


In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 134)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.89684


In [25]:
model.compile(optimizer='adam', loss=loss)

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [27]:
EPOCHS=30

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
786/786 [==============================] - 1089s 1s/step - loss: 2.0479
Epoch 2/30
786/786 [==============================] - 961s 1s/step - loss: 1.5888
Epoch 3/30
786/786 [==============================] - 918s 1s/step - loss: 1.4985
Epoch 4/30
786/786 [==============================] - 947s 1s/step - loss: 1.4518
Epoch 5/30
786/786 [==============================] - 922s 1s/step - loss: 1.4203
Epoch 6/30
786/786 [==============================] - 942s 1s/step - loss: 1.3953
Epoch 7/30
786/786 [==============================] - 972s 1s/step - loss: 1.3744
Epoch 8/30
786/786 [==============================] - 913s 1s/step - loss: 1.3559
Epoch 9/30
786/786 [==============================] - 951s 1s/step - loss: 1.3397
Epoch 10/30
786/786 [==============================] - 905s 1s/step - loss: 1.3253
Epoch 11/30
786/786 [==============================] - 896s 1s/step - loss: 1.3123
Epoch 12/30
786/786 [==============================] - 931s 1s/step - loss: 1.3017
Epoch 13/30


In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            34304     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 134)            137350    
Total params: 4,109,958
Trainable params: 4,109,958
Non-trainable params: 0
_________________________________________________________________


In [32]:
def generate_text(model, temperature, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
word_list = open("pastoral_word_seeds.txt", "r")

In [34]:
# print(generate_text(model, start_string=u" "))
for i in range(1,25001):    
    random.seed()
    
    filename = "samples/" + str(i) + ".txt"
    cur_temp = random.randint(1,100)/100
    
    seed_word = word_list.readline().strip() + " "
    
    print(filename + " " + str(cur_temp))
    file = open(filename, "w")
    
    file.write("This sample's Temperature: " + str(cur_temp) + "\n")
    file.write(generate_text(model, cur_temp, start_string=str(seed_word)))
    file.close()

samples/1.txt 0.86
samples/2.txt 0.68
samples/3.txt 0.21
samples/4.txt 0.98
samples/5.txt 0.77
samples/6.txt 0.66
samples/7.txt 0.71
samples/8.txt 0.53
samples/9.txt 0.34
samples/10.txt 0.13
samples/11.txt 0.47
samples/12.txt 0.2
samples/13.txt 0.05
samples/14.txt 0.86
samples/15.txt 0.65
samples/16.txt 0.72
samples/17.txt 0.76
samples/18.txt 0.76
samples/19.txt 0.1
samples/20.txt 0.48
samples/21.txt 0.17
samples/22.txt 0.85
samples/23.txt 0.44
samples/24.txt 0.39
samples/25.txt 0.19
samples/26.txt 0.78
samples/27.txt 0.45
samples/28.txt 0.97
samples/29.txt 0.36
samples/30.txt 0.38
samples/31.txt 0.47
samples/32.txt 0.87
samples/33.txt 0.96
samples/34.txt 0.87
samples/35.txt 0.49
samples/36.txt 0.05
samples/37.txt 0.51
samples/38.txt 0.88
samples/39.txt 0.62
samples/40.txt 0.9
samples/41.txt 0.41
samples/42.txt 0.6
samples/43.txt 0.74
samples/44.txt 0.6
samples/45.txt 0.69
samples/46.txt 0.82
samples/47.txt 0.56
samples/48.txt 0.35
samples/49.txt 0.58
samples/50.txt 0.14
samples/51.txt

samples/398.txt 0.31
samples/399.txt 0.76
samples/400.txt 0.91
samples/401.txt 0.44
samples/402.txt 0.04
samples/403.txt 0.02
samples/404.txt 0.58
samples/405.txt 0.39
samples/406.txt 0.45
samples/407.txt 0.14
samples/408.txt 0.92
samples/409.txt 0.41
samples/410.txt 0.92
samples/411.txt 0.33
samples/412.txt 0.74
samples/413.txt 0.67
samples/414.txt 0.29
samples/415.txt 0.12
samples/416.txt 0.15
samples/417.txt 0.56
samples/418.txt 0.24
samples/419.txt 0.77
samples/420.txt 0.97
samples/421.txt 0.38
samples/422.txt 0.42
samples/423.txt 0.94
samples/424.txt 0.56
samples/425.txt 0.91
samples/426.txt 0.95
samples/427.txt 0.63
samples/428.txt 0.35
samples/429.txt 0.49
samples/430.txt 0.89
samples/431.txt 0.9
samples/432.txt 0.63
samples/433.txt 0.11
samples/434.txt 0.55
samples/435.txt 0.95
samples/436.txt 0.46
samples/437.txt 0.97
samples/438.txt 0.97
samples/439.txt 0.24
samples/440.txt 0.36
samples/441.txt 0.41
samples/442.txt 0.54
samples/443.txt 0.73
samples/444.txt 0.36
samples/445.tx

KeyboardInterrupt: 